# 论文

标题：MFC到PyQt应用的迁移

## 概述

MFC是比较旧的一套界面库。随着技术的发展，出现了新一代界面库-Qt。它具有很多优点。但因为由C++编写，需要一定技术基础才能使用。Python是易于使用的解释型语言。PyQt框架结合的Python的易用和Qt的强大，具有很好的应用前景。本文结合实际项目，从多个方面论述了如何将一个MFC应用移植到PyQt框架，并进行了总结。

## 背景

C是一个通用的过程式语言, 它具有静态类型, 支持结构化编程[ref](https://en.wikipedia.org/wiki/C_(programming_language)). 随着时间的演变, C语言成为了程序间的通用语言, 每个编程语言必须支持C才能与系统交互. C语言是如此的旧, 存在各种遗留语法和默认标准, 以至于添加新功能变得十分困难--因为很多程序已经默认它不会改变. 事实上, 只有编译器才能解析C语言程序; 并且, C语言没有固定的ABI或内存布局[ref](https://gankra.github.io/blah/c-isnt-a-language/).

C++作为C的扩展, 加入了对象和标准库[ref](https://en.wikipedia.org/wiki/C%2B%2B). MFC用C++的形式封装了Windows API, 于1992年发布, 作为Windows界面编程的事实标准.

随着时间的发展, 出现了其他界面框架, 如wxWidgets和Qt. Qt4.0在2005年发布, 同时支持Windows和Linux. 它的消息系统使它广受欢迎.

Python是一个高阶的通用编程语言, 具有动态类型和自动垃圾回收, 使用Python编写的程序具有清晰的逻辑和很好的可读性. PyQt以插件的形式, 使人们在Python中可以使用Qt库. 

## 旧的项目

此项目为录像应用, 使用MFC/C++编写, VS2010开发, 运行于车载Windows7系统. 具有录像和记录运行信息功能.

录像功能使用FFmpeg库实现, 包含自定义的调用FFmpeg API的函数. 视频浏览使用摄像头厂家提供的SDK. 机车运行信息由连接到机车设备上的串口读取. 传感器信息由连接到传感器的串口读取. 机车两端的程序通过交换机连接, 通过socket通信.

## 移植过程

视频浏览库需要封装以供Python使用, 录像功能的依赖库过旧需要替代. socket通信使用网络服务替代. 串口由自定义封装.

### 底层库封装

使用boost-asio重写串口库；参考相关文档，将视频预览过程独立出来；最后使用boost-python创建python接口。

### 客户端相互通信

使用fastapi搭建小型服务器，通过访问服务器来获取数据。数据采用json格式。

### 录像功能

通过查找FFmpeg相关文档，得知使用独立FFmpeg程序，配合相关参数，可以实现相同功能，不必调用底层库。

## 功能实现

### 界面设计

整体界面为一个多选项卡的界面。

视频选项卡的构建通过编写视频窗口类，然后加入网格布局。通过监听子窗口的双击事件，隐藏或显示其他窗口，来实现窗口的最大化和最小化。

设备界面选项卡由几个表格组成。通过监听相关消息来更新表中显示的文字。其他选项卡与此类似。

### 数据保存

视频数据由FFmpeg进程保存，并自动切分。机车运行数据由串口读取，存储为二进制文件，放在每天的目录中。

### 数据的使用

替换硬盘后，使用配套播放器通过vlc播放视频，并读取二进制文件显示运行信息。

## 文档

使用Sphinx框架编写文档，并自动生成API文档。

## 测试

### 视频流测试

使用vlc与python绑定python-vlc，播放本地视频，模拟摄像头SDK。

### 设备模拟

通过使用diskpart创建虚拟磁盘，通过磁盘的挂载与卸载来模拟硬盘和U盘。

## 工具链

使用Git管理源代码，GitHub Actions构建C++插件和文档。

使用VSCode进行开发。配合插件：Python、C/C++、CMake Tools。

## 总结

本论文针对原有MFC程序的功能，探讨了将各个部分的迁移方法，并得出了迁移后的程序原型。新的程序具有易于修改使用和易于移植的优点。